# ML Model Training - EV Charging Demand Prediction

**MA EV ChargeMap Portfolio Project**

This notebook trains a machine learning model to predict daily charging demand (kWh) for candidate EV charging sites.

## Goals
1. Prepare features and target variable
2. Train multiple regression models
3. Evaluate model performance
4. Analyze feature importance
5. Export trained model for API deployment

In [ ]:
# Import libraries
import sys
sys.path.append('../backend')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import pickle

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

%matplotlib inline

## 1. Load and Prepare Data

In [ ]:
# Connect to database
from app.config import settings

engine = create_engine(settings.database_url)

# Load sites data
df = pd.read_sql("SELECT * FROM sites WHERE city = 'worcester'", engine)

print(f"Loaded {len(df)} sites")
df.head()

In [ ]:
# Define feature columns
feature_cols = [
    'traffic_index',
    'pop_density_index',
    'renters_share',
    'income_index',
    'poi_index',
    'parking_lot_flag',
    'municipal_parcel_flag'
]

# Target variable
target_col = 'daily_kwh_estimate'

# Prepare features and target
X = df[feature_cols].values
y = df[target_col].values

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nTarget statistics:")
print(f"  Min: {y.min():.1f} kWh")
print(f"  Max: {y.max():.1f} kWh")
print(f"  Mean: {y.mean():.1f} kWh")
print(f"  Std: {y.std():.1f} kWh")

## 2. Train-Test Split

In [ ]:
# Split data (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

## 3. Train Multiple Models

We'll compare three regression algorithms:
1. Linear Regression (baseline)
2. Random Forest Regressor
3. Gradient Boosting Regressor

In [ ]:
# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

# Train and evaluate each model
results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Evaluate
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    results[name] = {
        'model': model,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'predictions': y_pred_test
    }
    
    print(f"  Train R²: {train_r2:.4f}")
    print(f"  Test R²: {test_r2:.4f}")
    print(f"  Test RMSE: {test_rmse:.2f} kWh")
    print(f"  Test MAE: {test_mae:.2f} kWh")

## 4. Model Comparison

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Train R²': [r['train_r2'] for r in results.values()],
    'Test R²': [r['test_r2'] for r in results.values()],
    'Test RMSE': [r['test_rmse'] for r in results.values()],
    'Test MAE': [r['test_mae'] for r in results.values()],
})

print("\n=== Model Comparison ===")
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² scores
axes[0].bar(comparison_df['Model'], comparison_df['Test R²'], color='steelblue')
axes[0].set_ylabel('R² Score')
axes[0].set_title('Model Comparison: R² Score (Higher is Better)')
axes[0].set_ylim([0, 1])
for i, v in enumerate(comparison_df['Test R²']):
    axes[0].text(i, v + 0.02, f'{v:.3f}', ha='center')

# RMSE
axes[1].bar(comparison_df['Model'], comparison_df['Test RMSE'], color='coral')
axes[1].set_ylabel('RMSE (kWh)')
axes[1].set_title('Model Comparison: RMSE (Lower is Better)')
for i, v in enumerate(comparison_df['Test RMSE']):
    axes[1].text(i, v + 1, f'{v:.1f}', ha='center')

plt.tight_layout()
plt.show()

## 5. Select Best Model

We'll select the model with the best test R² score.

In [ ]:
# Select best model based on test R²
best_model_name = max(results.keys(), key=lambda k: results[k]['test_r2'])
best_model = results[best_model_name]['model']

print(f"Best model: {best_model_name}")
print(f"Test R²: {results[best_model_name]['test_r2']:.4f}")
print(f"Test RMSE: {results[best_model_name]['test_rmse']:.2f} kWh")

## 6. Prediction Visualization

In [ ]:
# Scatter plot of predictions vs actual
y_pred_best = results[best_model_name]['predictions']

plt.figure(figsize=(10, 8))
plt.scatter(y_test, y_pred_best, alpha=0.6, s=50)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Daily kWh')
plt.ylabel('Predicted Daily kWh')
plt.title(f'{best_model_name}: Predictions vs Actual', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Residual plot
residuals = y_test - y_pred_best

plt.figure(figsize=(10, 6))
plt.scatter(y_pred_best, residuals, alpha=0.6, s=50)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Daily kWh')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residual Plot', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Feature Importance

For tree-based models, we can analyze which features are most important for predictions.

In [ ]:
# Feature importance (if available)
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    
    # Create dataframe
    importance_df = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    print("\n=== Feature Importance ===")
    print(importance_df.to_string(index=False))
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['Feature'], importance_df['Importance'], color='steelblue')
    plt.xlabel('Importance')
    plt.title(f'{best_model_name}: Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print(f"\n{best_model_name} does not provide feature importances.")
    print("For linear models, check coefficients instead.")
    
    if hasattr(best_model, 'coef_'):
        coef_df = pd.DataFrame({
            'Feature': feature_cols,
            'Coefficient': best_model.coef_
        }).sort_values('Coefficient', key=abs, ascending=False)
        
        print("\n=== Model Coefficients ===")
        print(coef_df.to_string(index=False))

## 8. Cross-Validation

Verify model stability with cross-validation.

In [ ]:
# 5-fold cross-validation on best model
print(f"Running 5-fold cross-validation for {best_model_name}...\n")

cv_scores = cross_val_score(best_model, X, y, cv=5, 
                           scoring='r2', n_jobs=-1)

print(f"Cross-validation R² scores: {cv_scores}")
print(f"Mean CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Visualize CV scores
plt.figure(figsize=(8, 5))
plt.bar(range(1, 6), cv_scores, color='steelblue', alpha=0.7)
plt.axhline(cv_scores.mean(), color='red', linestyle='--', 
           label=f'Mean: {cv_scores.mean():.3f}')
plt.xlabel('Fold')
plt.ylabel('R² Score')
plt.title('Cross-Validation Scores', fontsize=14, fontweight='bold')
plt.legend()
plt.ylim([0, 1])
plt.tight_layout()
plt.show()

## 9. Export Model

In [ ]:
# Train final model on full dataset
print("Training final model on full dataset...")
final_model = type(best_model)(**best_model.get_params())
final_model.fit(X, y)

# Export model
model_path = '../models/site_score_model.pkl'

with open(model_path, 'wb') as f:
    pickle.dump(final_model, f)

print(f"\n✓ Model saved to {model_path}")
print(f"\nModel details:")
print(f"  Type: {type(final_model).__name__}")
print(f"  Features: {len(feature_cols)}")
print(f"  Training samples: {len(X)}")
print(f"  Expected R² on new data: ~{cv_scores.mean():.3f}")

## 10. Model Testing

Test the exported model to ensure it loads correctly.

In [ ]:
# Load exported model
with open(model_path, 'rb') as f:
    loaded_model = pickle.load(f)

# Test prediction
test_features = np.array([[
    0.7,  # traffic_index
    0.6,  # pop_density_index
    0.5,  # renters_share
    0.4,  # income_index
    0.65, # poi_index
    1,    # parking_lot_flag
    0     # municipal_parcel_flag
]])

prediction = loaded_model.predict(test_features)[0]

print(f"\n=== Model Test ===")
print(f"Test input features: {test_features[0]}")
print(f"Predicted daily kWh: {prediction:.1f}")
print(f"\n✓ Model loads and predicts successfully!")

## Summary

### Model Performance

We trained and compared three regression models for predicting daily EV charging demand:

1. **Linear Regression**: Baseline model
2. **Random Forest**: Ensemble tree-based model
3. **Gradient Boosting**: Advanced ensemble model

The best model was selected based on test set R² score and exported for deployment.

### Key Features

The most important features for predicting charging demand are:
- Traffic index
- Population density
- Points of interest index

### Deployment

The trained model is saved as a pickle file and can be loaded by the FastAPI backend to provide real-time predictions via the `/api/predict` endpoint.

### Limitations & Future Work

1. **Data**: Model trained on synthetic/simulated data. In production, would use real charger usage data.
2. **Features**: Could add temporal features (time of day, day of week) for better predictions.
3. **Validation**: With real data, could validate against actual charger utilization.
4. **Model Updates**: Should retrain periodically as new data becomes available.